In [ ]:
pip install deep-translator sacrebleu datasets #install if needed

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 20.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requi

In [ ]:
from google.colab import drive
drive.mount('/content/drive') # mount drive

Mounted at /content/drive


In [ ]:
import os
from deep_translator import GoogleTranslator
import sacrebleu
from datasets import Dataset
import xml.etree.ElementTree as ET
# initialize dataset as in NLLB
base_path = "/content/drive/MyDrive/Colab Notebooks/CompLing"
en_devtest_path = os.path.join(base_path, "eng_Latn.devtest")
gn_devtest_path = os.path.join(base_path, "grn_Latn.devtest")
# get just the flores data
def load_flores_for_bleu(english_path: str, guarani_path: str) -> Dataset:
    with open(english_path, encoding="utf-8") as f_en:
        en_lines = [line.strip() for line in f_en if line.strip()]
    with open(guarani_path, encoding="utf-8") as f_gn:
        gn_lines = [line.strip() for line in f_gn if line.strip()]
    if len(en_lines) != len(gn_lines):
        raise ValueError(f"Line mismatch: {len(en_lines)} English vs {len(gn_lines)} Guarani.")
    data = [{"translation": {"en": en, "gn": gn}} for en, gn in zip(en_lines, gn_lines)]
    return Dataset.from_list(data)

def extract_translation_columns(dataset):
    return dataset.map(lambda x: {
        "en": x["translation"]["en"],
        "gn": x["translation"]["gn"]
    })
# normalize function to match google translate's output
def normalize_glottal_stop(text):
    return (
        text
        .replace("\\'", "ʼ")
        .replace("'", "ʼ")
        .replace("’", "ʼ")
    )

def normalize_dataset(dataset):
    return dataset.map(lambda x: {
        "en": x["en"],
        "gn": normalize_glottal_stop(x["gn"])
    })
# get dataset for translation
test_dataset = load_flores_for_bleu(en_devtest_path, gn_devtest_path)
test_dataset = extract_translation_columns(test_dataset)
test_dataset = test_dataset.select(range(100))
test_dataset = normalize_dataset(test_dataset)
source_sentences = test_dataset["en"]
reference_sentences = [normalize_glottal_stop(t) for t in test_dataset["gn"]]

translator = GoogleTranslator(source='en', target='gn')
translated_sentences = []
# translation loop for evaluation
for idx, sentence in enumerate(source_sentences, 1):
    try:
        translation = translator.translate(sentence)
        translated_sentences.append(normalize_glottal_stop(translation))
        print(f"Translated {idx}/{len(source_sentences)}", end='\r')
    except Exception as e:
        print(f"\nError translating: {sentence} -> {e}")
        translated_sentences.append("")

# Eval
chrf_score = sacrebleu.corpus_chrf(translated_sentences, [reference_sentences], word_order=2)
print(f"\nChrF++ score for Google Translate: {chrf_score.score:.2f}")

Map:   0%|          | 0/1012 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Translated 100/100
ChrF++ score for Google Translate: 32.24
